<a href="https://colab.research.google.com/github/chouhandiksha/bigdataproject/blob/colab/notebooks/diksha/analyse_NY_mobility_whitepop_time_series_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Spark SQL Documentation:** 
https://spark.apache.org/docs/2.2.0/sql-programming-guide.html

In [1]:
# add time information at the end of every cell
!pip install ipython-autotime
%load_ext autotime

time: 1.94 ms (started: 2021-04-16 04:45:08 +00:00)


In [2]:
# Install required dependancies
!pip install pyspark
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 212.3MB 69kB/s 
     |████████████████████████████████| 204kB 38.9MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=7dcbb6badba801a7cc3d6df697cb628362529f58cb04fb0246f74a8354685c71
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 31 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading

In [3]:
# Import modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

time: 92.9 ms (started: 2021-04-16 04:46:00 +00:00)


In [4]:
import altair as alt
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

time: 283 ms (started: 2021-04-16 04:46:01 +00:00)


In [5]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

time: 4.93 s (started: 2021-04-16 04:46:01 +00:00)


In [6]:
#sc.stop()

time: 1.45 ms (started: 2021-04-16 04:46:06 +00:00)


In [7]:
# Mount drive with data
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
time: 24.7 s (started: 2021-04-16 04:46:06 +00:00)


In [8]:
# Set path to data folder
path = Path('drive/MyDrive/big-data-project/data/clean-data')
city = 'ny'

time: 3.26 ms (started: 2021-04-16 04:46:31 +00:00)


In [9]:
# Read data into dataframe
df_soc = spark.read.format('csv').option('header','true').option('quote',"\"").option('escape',"\"").load(str(path/city/'social/2020/*.csv'))
df_soc.show()

+---+------------+--------------------+--------------------+------------+---------------------------+--------------------------+------------------------------------------+----------------------------+----------------------+------------------------+--------------------+-------------------------------+-------------------------------+--------------------+-------------------------+--------------------------+----------------------+----------------------------+---------------------------+-----------------------------+
|_c0|         cbg|    date_range_start|      date_range_end|device_count|distance_traveled_from_home|bucketed_distance_traveled|median_dwell_at_bucketed_distance_traveled|completely_home_device_count|median_home_dwell_time|bucketed_home_dwell_time|at_home_by_each_hour|part_time_work_behavior_devices|full_time_work_behavior_devices|    destination_cbgs|delivery_behavior_devices|median_non_home_dwell_time|candidate_device_count|bucketed_away_from_home_time|median_percentage_time_

In [10]:
# View schema
df_soc.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- cbg: string (nullable = true)
 |-- date_range_start: string (nullable = true)
 |-- date_range_end: string (nullable = true)
 |-- device_count: string (nullable = true)
 |-- distance_traveled_from_home: string (nullable = true)
 |-- bucketed_distance_traveled: string (nullable = true)
 |-- median_dwell_at_bucketed_distance_traveled: string (nullable = true)
 |-- completely_home_device_count: string (nullable = true)
 |-- median_home_dwell_time: string (nullable = true)
 |-- bucketed_home_dwell_time: string (nullable = true)
 |-- at_home_by_each_hour: string (nullable = true)
 |-- part_time_work_behavior_devices: string (nullable = true)
 |-- full_time_work_behavior_devices: string (nullable = true)
 |-- destination_cbgs: string (nullable = true)
 |-- delivery_behavior_devices: string (nullable = true)
 |-- median_non_home_dwell_time: string (nullable = true)
 |-- candidate_device_count: string (nullable = true)
 |-- bucketed_away_from_home_ti

In [11]:
# # Take small sample of data to experiment with
# df_soc = df_soc.limit(100)
# df_soc.show()

time: 1.19 ms (started: 2021-04-16 04:46:50 +00:00)


In [12]:
df_soc.createOrReplaceTempView('clean_ny')
df_soc = spark.sql('SELECT cbg, date_range_start as date, device_count, completely_home_device_count, part_time_work_behavior_devices, full_time_work_behavior_devices FROM clean_ny WHERE device_count > 5')
df_soc.show()

+------------+--------------------+------------+----------------------------+-------------------------------+-------------------------------+
|         cbg|                date|device_count|completely_home_device_count|part_time_work_behavior_devices|full_time_work_behavior_devices|
+------------+--------------------+------------+----------------------------+-------------------------------+-------------------------------+
|360470064002|2020-02-03T00:00:...|          64|                          18|                              7|                              5|
|360810384001|2020-02-03T00:00:...|          84|                          21|                             12|                              8|
|360850170103|2020-02-03T00:00:...|         250|                          68|                             41|                             26|
|360050213021|2020-02-03T00:00:...|          92|                          26|                             11|                              6|
|36005

In [13]:
# View schema
df_soc.printSchema()

root
 |-- cbg: string (nullable = true)
 |-- date: string (nullable = true)
 |-- device_count: string (nullable = true)
 |-- completely_home_device_count: string (nullable = true)
 |-- part_time_work_behavior_devices: string (nullable = true)
 |-- full_time_work_behavior_devices: string (nullable = true)

time: 10.9 ms (started: 2021-04-16 04:46:51 +00:00)


In [14]:
# Add completely home percentage column
df_soc = df_soc.withColumn('completely_home_percentage', (df_soc['completely_home_device_count']/df_soc['device_count']) * 100)
df_soc = df_soc.withColumn('part_time_work_percentage', (df_soc['part_time_work_behavior_devices']/df_soc['device_count']) * 100)
df_soc = df_soc.withColumn('full_time_work_percentage', (df_soc['full_time_work_behavior_devices']/df_soc['device_count']) * 100)

df_soc.show()

+------------+--------------------+------------+----------------------------+-------------------------------+-------------------------------+--------------------------+-------------------------+-------------------------+
|         cbg|                date|device_count|completely_home_device_count|part_time_work_behavior_devices|full_time_work_behavior_devices|completely_home_percentage|part_time_work_percentage|full_time_work_percentage|
+------------+--------------------+------------+----------------------------+-------------------------------+-------------------------------+--------------------------+-------------------------+-------------------------+
|360470064002|2020-02-03T00:00:...|          64|                          18|                              7|                              5|                    28.125|                  10.9375|                   7.8125|
|360810384001|2020-02-03T00:00:...|          84|                          21|                             12|       

In [15]:
df_soc.printSchema()

root
 |-- cbg: string (nullable = true)
 |-- date: string (nullable = true)
 |-- device_count: string (nullable = true)
 |-- completely_home_device_count: string (nullable = true)
 |-- part_time_work_behavior_devices: string (nullable = true)
 |-- full_time_work_behavior_devices: string (nullable = true)
 |-- completely_home_percentage: double (nullable = true)
 |-- part_time_work_percentage: double (nullable = true)
 |-- full_time_work_percentage: double (nullable = true)

time: 11.5 ms (started: 2021-04-16 04:46:52 +00:00)


In [16]:
#Create temp view
df_soc.createOrReplaceTempView('mobility')

time: 39.4 ms (started: 2021-04-16 04:46:52 +00:00)




---


**Demographic Data**






---



In [17]:
# Read poverty data
# Read data into RDD
df_demographic = spark.read.format('csv').option('header','true').option('quote',"\"").option('escape',"\"").load(str(path/city/'ny.csv'))
df_demographic.createOrReplaceTempView('demographic')
df_demographic.show()

+------------+---------+------------------+------------------+
|         cbg|pop_total|poverty_percentage|    perc_whiteonly|
+------------+---------+------------------+------------------+
|360050001000|        0|               0.0|               0.0|
|360050001001|     7503|               0.0|12.435025989604158|
|360050002000|        0|               0.0|               0.0|
|360050002001|     2114|16.130558183538316|35.856196783349105|
|360050002002|     2168|18.911439114391143| 48.06273062730627|
|360050002003|      969| 8.152734778121776|49.742002063983485|
|360050004000|        0|               0.0|               0.0|
|360050004001|      646|32.972136222910216| 44.27244582043344|
|360050004002|     1400|25.571428571428573| 75.35714285714286|
|360050004003|     3393| 1.569905213270142| 33.65753020925435|
|360050004004|      541|15.196998123827393| 20.33271719038817|
|360050016001|     3427|  25.1531952144733|38.400933761307265|
|360050016002|      662| 62.99093655589124| 73.56495468

In [18]:
df_mob_demo = spark.sql('SELECT mobility.*, demographic.perc_whiteonly from mobility INNER JOIN demographic ON mobility.cbg = demographic.cbg')
df_mob_demo.createOrReplaceTempView('demographic_mobility')
df_mob_demo.show()


+------------+--------------------+------------+----------------------------+-------------------------------+-------------------------------+--------------------------+-------------------------+-------------------------+------------------+
|         cbg|                date|device_count|completely_home_device_count|part_time_work_behavior_devices|full_time_work_behavior_devices|completely_home_percentage|part_time_work_percentage|full_time_work_percentage|    perc_whiteonly|
+------------+--------------------+------------+----------------------------+-------------------------------+-------------------------------+--------------------------+-------------------------+-------------------------+------------------+
|360470064002|2020-02-03T00:00:...|          64|                          18|                              7|                              5|                    28.125|                  10.9375|                   7.8125| 92.27053140096618|
|360810384001|2020-02-03T00:00:...|     

In [19]:
# DIVIDING THE PEOPLE TOP 20 PERCENT AND BOTTOM 20 PERCENT

df_mob_demo_high_class =  spark.sql('SELECT * FROM demographic_mobility WHERE perc_whiteonly > 80')
grouped_df_mob_demo_high_class = df_mob_demo_high_class.groupBy("date").mean("completely_home_percentage").withColumnRenamed('avg(completely_home_percentage)','completely_home_percentage')
grouped_df_mob_demo_high_class.createOrReplaceTempView('mob_demo_white_majority')


df_mob_demo_low_class =  spark.sql('SELECT * FROM demographic_mobility WHERE perc_whiteonly < 20')
grouped_df_mob_demo_low_class = df_mob_demo_low_class.groupBy("date").mean("completely_home_percentage").withColumnRenamed('avg(completely_home_percentage)','completely_home_percentage')
grouped_df_mob_demo_low_class.createOrReplaceTempView('mob_demo_white_minority')



grouped_df_mob_demo_combined = spark.sql("""

SELECT wh.date,wh.completely_home_percentage as high_comp_home_perc,
wl.completely_home_percentage as low_comp_home_perc  
FROM mob_demo_white_majority wh INNER JOIN mob_demo_white_minority wl ON wh.date = wl.date

""")


time: 324 ms (started: 2021-04-16 04:46:54 +00:00)


In [20]:
#grouped_df_mob_demo_combined.show()


time: 2.92 ms (started: 2021-04-16 04:46:54 +00:00)


In [21]:
grouped_df_mob_demo_combined_pandas = grouped_df_mob_demo_combined.toPandas()
grouped_df_mob_demo_combined_pandas

,date,high_comp_home_perc,low_comp_home_perc
0,2020-02-26T00:00:00-05:00,18.919700,24.121034
1,2020-07-03T00:00:00-04:00,31.722549,39.530227
2,2020-10-18T00:00:00-04:00,33.102480,41.845833
3,2020-07-14T00:00:00-04:00,31.602094,39.190459
4,2020-03-01T00:00:00-05:00,30.464927,38.615294
...,...,...,...
361,2020-04-16T00:00:00-04:00,50.336986,55.051239
362,2020-04-30T00:00:00-04:00,45.399188,51.142434
363,2020-06-14T00:00:00-04:00,36.314660,45.949476
364,2020-12-14T00:00:00-05:00,37.765826,41.079958


time: 2min 44s (started: 2021-04-16 04:46:54 +00:00)


---
**Graph** 

---

In [22]:
base = alt.Chart(grouped_df_mob_demo_combined_pandas.reset_index()).encode(x='date')

alt.layer(
    base.mark_line(color='blue').encode(y='high_comp_home_perc'),
    
    base.mark_line(color='red').encode(y='low_comp_home_perc')

)

alt.LayerChart(...)

time: 79.1 ms (started: 2021-04-16 04:49:38 +00:00)
